<a href="https://colab.research.google.com/github/hara-ginta/anogan/blob/master/python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/My Drive/大学/異常検知研究/anogan


/content/drive/My Drive/大学/異常検知研究/anogan


In [4]:
from dcgan import DCGAN
import numpy as np
from keras.optimizers import Adam
from data_loader import load_cucumber


def normalize(X):
    return (X-127.5)/127.5


def denormalize(X):
    return ((X + 1.0)/2.0*255.0).astype(dtype=np.uint8)


if __name__ == '__main__':
    batch_size = 16
    epochs = 4000
    input_dim = 30
    g_optim = Adam(lr=0.0001, beta_1=0.5, beta_2=0.9)
    d_optim = Adam(lr=0.0001, beta_1=0.5, beta_2=0.9)

    ### 0. prepare data
    X_train, X_test, y_test = load_cucumber()
    X_train = normalize(X_train)
    X_test = normalize(X_test)
    input_shape = X_train[0].shape
    X_test_original = X_test.copy()

    ### 1. train generator & discriminator
    dcgan = DCGAN(input_dim, input_shape)
    dcgan.compile(g_optim, d_optim)
    g_losses, d_losses = dcgan.train(epochs, batch_size, X_train)
    with open('loss.csv', 'w') as f:
        for g_loss, d_loss in zip(g_losses, d_losses):
            f.write(str(g_loss) + ',' + str(d_loss) + '\n')

ストリーミング出力は最後の 5000 行に切り捨てられました。
Epoch1500 end
15/16 [===========================>..] - ETA: 0s - g: 2.1288 - d: 0.2724
Epoch1501 end
14/16 [=========================>....] - ETA: 0s - g: 2.0147 - d: 0.2667
Epoch1502 end
15/16 [===========================>..] - ETA: 0s - g: 2.0087 - d: 0.2761
Epoch1503 end
14/16 [=========================>....] - ETA: 0s - g: 2.1276 - d: 0.2846
Epoch1504 end
15/16 [===========================>..] - ETA: 0s - g: 2.0529 - d: 0.2659
Epoch1505 end
14/16 [=========================>....] - ETA: 0s - g: 2.1651 - d: 0.2823
Epoch1506 end
15/16 [===========================>..] - ETA: 0s - g: 2.1621 - d: 0.2660
Epoch1507 end
14/16 [=========================>....] - ETA: 0s - g: 2.0833 - d: 0.2864
Epoch1508 end
15/16 [===========================>..] - ETA: 0s - g: 2.1366 - d: 0.2665
Epoch1509 end
15/16 [===========================>..] - ETA: 0s - g: 2.0845 - d: 0.2810
Epoch1510 end
14/16 [=========================>....] - ETA: 0s - g: 2.2198 - d: 0.2701
Epoch1511 e

In [5]:
import os, cv2
from dcgan import DCGAN
from anogan import ANOGAN
import numpy as np
from keras.optimizers import Adam
from data_loader import load_cucumber
from train import normalize, denormalize


if __name__ == '__main__':
    iterations = 100
    input_dim = 30
    anogan_optim = Adam(lr=0.001, amsgrad=True)

    ### 0. prepare data
    X_train, X_test, y_test = load_cucumber()
    X_train = normalize(X_train)
    X_test = normalize(X_test)
    input_shape = X_train[0].shape

    ### 1. train generator & discriminator
    dcgan = DCGAN(input_dim, input_shape)
    dcgan.load_weights('weights/generator_3999.h5', 'weights/discriminator_3999.h5')

    for i, test_img in enumerate(X_test):
        test_img = test_img[np.newaxis,:,:,:]
        anogan = ANOGAN(input_dim, dcgan.g)
        anogan.compile(anogan_optim)
        anomaly_score, generated_img = anogan.compute_anomaly_score(test_img, iterations)
        generated_img = denormalize(generated_img)
        imgs = np.concatenate((denormalize(test_img[0]), generated_img[0]), axis=1)
        cv2.imwrite('predict' + os.sep + str(int(anomaly_score)) + '_' + str(i) + '.png', imgs)
        print(str(i) + ' %.2f'%anomaly_score)
        with open('scores.txt', 'a') as f:
            f.write(str(anomaly_score) + '\n')

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
0 206.21
1 205.31
2 134.42
3 210.82
4 191.83
5 177.36
6 572.18
7 411.98
8 271.07
9 211.26
10 152.36
11 152.40
12 208.15
13 193.39
14 207.19
15 391.44
16 610.80
17 143.39
18 280.13
19 138.26
20 173.04
21 517.02
22 239.98
23 138.60
24 273.76
25 303.01
26 170.35
27 323.53
28 188.02
29 148.96
30 142.32
31 298.23
32 179.91
33 181.99
34 380.33
35 274.08
36 113.92
37 155.41
38 216.09
39 598.86
40 161.17
41 251.69
42 265.57
43 638.05
44 168.78
45 370.78
46 219.20
47 522.31
48 214.99
49 182.49
50 157.68
51 130.10
52 188.45
53 329.56
54 109.87
55 168.84
56 253.78
57 221.75
58 234.21
59 192.35
60 482.36
61 303.28
62 143.68
63 215.45
64 139.38
65 178.45
66 231.66
67 248.82
68 347.84
69 211.55
70 316.49
71 463.81
72 385.72
73 616.21
74 285.72
75 208.11
76 585.90
77 175.88
78 209.24
79 347.23
80 827.04
81 487.64
82 240.30
83 306.58
84 522.98
85 782.55
86 263.25
87 332

# 新しいセクション